# Training a YOLO implementation in pytorch

## Step 1: Preparation

### Upgrades

In [2]:
!pip install --upgrade sagemaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.8 MB/s eta 0:00:00
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.239.0
    Uninstalling sagemaker-2.239.0:
      Successfully uninstalled sagemaker-2.239.0


### Import requirements

In [10]:
import os
import boto3
import sagemaker
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.pytorch import PyTorch
from sagemaker.debugger import DebuggerHookConfig, CollectionConfig

In [5]:
VOC_TAR = "VOCtrainval_11-May-2012.tar"
BUCKET_NAME = 'sagemaker-20250212110251'

### Initialize session and role

In [6]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

[02/18/25 17:04:36] INFO     Found credentials from IAM Role:                                   ]8;id=582772;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=917062;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=845877;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=61990;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

### S3 locations

In [8]:
raw_data_s3_uri = f's3://{BUCKET_NAME}/datasets/{VOC_TAR}'
processed_prefix = 'datasets/processed/VOC'  # Folder where processed data is stored
processed_data_s3_uri = f's3://{BUCKET_NAME}/{processed_prefix}'

## Step 2: Preprocessing Job

Create a ScriptProcessor to run a data preprocessing script.
In this example, "preprocess.py" should be a script you create that:
- Downloads and extracts the tar file.
- Splits the dataset into training/validation/test sets.
- Converts the dataset into a format (for example, preprocessed images and annotation files) that your training script expects.
Upload this script into a local folder (e.g., "preprocessing") which you'll point to as your source_dir.

In [18]:
image_uri = sagemaker.image_uris.retrieve(
    framework='pytorch',
    region=sagemaker_session.boto_region_name,
    version='2.1.0',
    py_version='py310',
    image_scope='training',
    instance_type='ml.m5.xlarge'
)

script_processor = ScriptProcessor(
    command=['python3'],
    image_uri=image_uri,
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    sagemaker_session=sagemaker_session
)

### Run the processing job

In [11]:
s3 = boto3.client('s3')

# List objects under the processed data prefix
response = s3.list_objects_v2(Bucket=BUCKET_NAME, Prefix=processed_prefix)
if 'Contents' in response and len(response['Contents']) > 0:
    print("Processed data already exists. Skipping processing job.")
else:
    print("Processed data not found. Running processing job.")
    script_processor.run(
        code='preprocess.py',
        source_dir='preprocessing',
        inputs=[
            ProcessingInput(
                source=raw_data_s3_uri,
                destination='/opt/ml/processing/input'
            )
        ],
        outputs=[
            ProcessingOutput(
                output_name='processed_data',
                source='/opt/ml/processing/output',
                destination=processed_data_s3_uri
            )
        ],
        arguments=[
            '--image_size', '448'
        ]
    )

    print("Data preprocessing complete. Processed data available at:", processed_data_s3_uri)

[02/18/25 17:07:25] INFO     Found credentials from IAM Role:                                   ]8;id=784819;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=938504;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Processed data not found. Running processing job.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7 else:                                                                                       │
│    8 │   print("Processed data not found. Running processing job.")                              │
│    9 │                                                                                           │
│ ❱ 10 │   image_uri = sagemaker.image_uris.retrieve(                                              │
│   11 │   │   framework='pytorch',                                                                │
│   12 │   │   region=sagemaker_session.boto_region_name,                                          │
│   13 │   │   version='2.1.0',                                                                    │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/workflow/utili │
│ ties.py:424 in wrapper                                                                           │
│                                                                                                  │
│   421 │   │   │   if isinstance(value, Parameter):                                               │
│   422 │   │   │   │   logger.warning(warning_msg_template, arg_name, func_name, type(value))     │
│   423 │   │   │   │   kwargs[arg_name] = value.default_value                                     │
│ ❱ 424 │   │   return func(*args, **kwargs)                                                       │
│   425 │                                                                                          │
│   426 │   return wrapper                                                                         │
│   427                                                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/image_uris.py: │
│ 196 in retrieve                                                                                  │
│                                                                                                  │
│   193 │   │   │   inference_tool = _get_inference_tool(inference_tool, instance_type)            │
│   194 │   │   │   if inference_tool in ["neuron", "neuronx"]:                                    │
│   195 │   │   │   │   _framework = f"{framework}-{inference_tool}"                               │
│ ❱ 196 │   │   final_image_scope = _get_final_image_scope(framework, instance_type, image_scope   │
│   197 │   │   _validate_for_suppported_frameworks_and_instance_type(framework, instance_type)    │
│   198 │   │   config = _config_for_framework_and_scope(_framework, final_image_scope, accelera   │
│   199                                                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/image_uris.py: │
│ 397 in _config_for_framework_and_scope                                                           │
│                                                                                                  │
│   394 │   │   │   "Same images used for training and inference. Defaulting to image scope: %s.   │
│   395 │   │   │   available_scopes[0],                                                           │
│   396 │   │   )                                                                                  │
│ ❱ 397 │   │   image_scope = available_scopes[0]            

## Step 2: Launch a Training Job with Debugger

### Define a Debugger hook configuration. This tells SageMaker which collections to capture.

In [ ]:
debugger_hook_config = PyTorchDebuggerHookConfig(
    collection_configs=[CollectionConfig(name="Losses")]
)

### Create a PyTorch estimator. Ensure your training code (e.g., train.py and supporting modules) is available in a source directory.

In [ ]:
estimator = PyTorch(
    entry_point='train.py',
    source_dir='training',  # directory containing train.py and other training modules
    role=role,
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    framework_version='1.8.1',
    py_version='py3',
    debugger_hook_config=debugger_hook_config,
    hyperparameters={
        'epochs': 10,
        'batch-size': 32,
        'learning-rate': 0.001,
        # add other hyperparameters as needed
        'data-dir': '/opt/ml/input/data/processed',
        'tar-file': ''  # Not needed in training if data is preprocessed
    },
    debugger_rule_configs=[
        # You can add built-in or custom debugger rules here.
    ]
)

### Define channel for preprocessed data.

The processed data is output from the processing job and will be used as input for training.

In [ ]:
data_channels = {
    'processed': processed_data_s3_uri
}

### Launch the training job

In [ ]:
estimator.fit(inputs=data_channels)